In [101]:
from tqdm.notebook import tqdm
from math import sqrt,erf
from multiprocessing import Pool
from itertools import product, repeat
from matplotlib import pyplot as plt
from matplotlib import animation
import matplotlib as mpl
import numpy as np
from varname import nameof
from qutip import *
from matplotlib.cm import get_cmap
import colorcet as cc
import subprocess
import os, shutil
import time
import functools
from scipy import special
import scipy
from IPython.display import display, clear_output


plt.style.use('ggplot')
markers = ["o", "X", "P", "p", "*"]
cols = [p['color'] for p in plt.rcParams['axes.prop_cycle']]
plt.rc('text.latex', preamble=r'\usepackage{amsmath}\usepackage{braket}\usepackage{nicefrac}')
plt.rcParams.update({'font.size': 30,
                     'figure.figsize': (11,7),
                     'axes.facecolor': 'white',
                     'axes.edgecolor': 'lightgray',
                     "figure.autolayout": 'True',
                     'axes.xmargin': 0.03,
                     'axes.ymargin': 0.05,
                     'axes.grid': False,
                     'axes.linewidth': 5,
                     'lines.markersize': 15,
                     'text.usetex': True,
                     'lines.linewidth': 8,
                     "legend.frameon": True,
                     "legend.framealpha": 0.7,
                     "legend.handletextpad": 1,
                     "legend.edgecolor": "black",
                     "legend.handlelength": 1,
                     "legend.labelspacing": 0,
                     "legend.columnspacing": 1,
                     "legend.fontsize": 35,
                    })
linestyles = ["-", "--", ":"]
bbox = dict(boxstyle="round", facecolor="lightgray")

D0 = 1
deltaD = 0.001
plt.plot([], [])
plt.show()
clear_output()

In [ ]:
def get_RG_flow(J0_by_D0, Ub_by_J=0, plot=False):
    """ Returns the flow of couplings in the form of two ndarrays J and D.
    Each ndarray is in ascending order of the bandwidth. """
    J0 = D0 * J0_by_D0
    Ub = - Ub_by_J * J0
    omega = -D0/2
    
    ### initialise arrays with UV values
    D = [D0]
    J = [J0]
    
    ### apply URG transformations until bandwith vanishes
    ### or J reduces to zero.
    while D[-1] >= deltaD and J[-1] >= 0:
        
        ### URG equation
        deltaJ = - J[-1] * (J[-1] + 4 * Ub) / (omega - D[-1]/2 + J[-1]/4) * deltaD
        
        ### Check if denominator has changed sign, 
        ### then append renormalised values to array
        if (omega - (D[-1] - deltaD)/2 + (J[-1] + deltaJ)/4) * (omega - D0/2 + J0/4) > 0:
            D.append(D[-1] - deltaD)
            J.append(J[-1] + deltaJ)
        else:
            break
    
    ### plot values
    if plot:
        plt.plot(np.array(D)/D0, np.array(J)/J0, marker="o")
        plt.xlabel(r"$D/D_0$")
        plt.ylabel(r"$J/J_0$")
    # print (len(J), D0/deltaD)
    return np.flip(D), np.flip(J)

_ = get_RG_flow(0.35, Ub_by_J=0, plot=True)
# print (D[0]/deltaD, J[0], D[0])

In [98]:
def fit_func(x, a, b, c, d, e):
    return a + b * x**c * np.exp(-d * x**e)

def get_renorm_per_comb(args_arr):
    iterables, new_combinations, new_coefficients, q_pos_init, alpha, ph_symmetrise_factor = args_arr
    new_coefficients = new_coefficients.tolist()
    # new_combinations = new_combinations.tolist()
    target_combs, target_renorms = [], []
    for source_comb, coeff in tqdm(iterables, disable=False):
        
        ## loop over spins of IOM, particle/hole sectors as well as cloud members
        q_pos_arr = [q_pos_init, q_pos_init + 2] if ph_symmetrise_factor == 2 else [q_pos_init]
        for q_pos, beta, k_pos in product(q_pos_arr, [1, -1], range(1, q_pos_init - 1, 2)):
            beta_pos = int((1 - beta) / 2)
            betabar_pos = int(1 - beta_pos)
            if  source_comb[k_pos + beta_pos] == 1 - source_comb[q_pos + beta_pos]:
                target_comb = np.copy(source_comb).tolist()
                target_comb[k_pos + beta_pos] = 1 - source_comb[k_pos + beta_pos]
                target_comb[q_pos + beta_pos] = 1 - source_comb[q_pos + beta_pos]
                exchange_sign = (-1) ** (1 - source_comb[q_pos + beta_pos]) * (-1) ** sum(source_comb[k_pos:q_pos])
                if target_comb in new_combinations:
                    new_coefficients[new_combinations.index(target_comb)] += exchange_sign * alpha * beta * (source_comb[0] - 0.5) * coeff
                else:
                    new_coefficients.append(exchange_sign * alpha * beta * (source_comb[0] - 0.5) * coeff)
                    new_combinations.append(target_comb)

            if  source_comb[k_pos + betabar_pos] == 1 - source_comb[q_pos + beta_pos] and source_comb[0] == 1 - abs((1 - beta)/2 - source_comb[q_pos + beta_pos]):
                target_comb = np.copy(source_comb).tolist()
                target_comb[k_pos + betabar_pos] = 1 - source_comb[k_pos + betabar_pos]
                target_comb[q_pos + beta_pos] = 1 - source_comb[q_pos + beta_pos]
                target_comb[0] = 1 - source_comb[0]
                exchange_sign = (-1) ** (1 - source_comb[q_pos + beta_pos]) * (-1) ** sum(source_comb[k_pos:q_pos])
                if target_comb in new_combinations:
                    new_coefficients[new_combinations.index(target_comb)] += exchange_sign * alpha * beta * (source_comb[0] - 0.5) * coeff
                else:
                    new_coefficients.append(exchange_sign * alpha * beta * (source_comb[0] - 0.5) * coeff)
                    new_combinations.append(target_comb)
    
    # for comb,renorm in tqdm(zip(target_combs, target_renorms), total=len(target_combs)):
    #     if comb not in new_combinations:
    #         new_combinations.append(comb)
    #         new_coefficients = np.append(new_coefficients, 0)
    #     new_coefficients[new_combinations.index(comb)] += renorm

    return new_combinations, new_coefficients


def get_renorm(coefficients, combinations, savepath, total_num, alpha, ph_symmetrise_factor):
    """ Calculates and returns the renormalised coefficients at a given RG step. """
    q_pos = total_num - 2 * ph_symmetrise_factor ### position of the nearest IOM
    try:
        ### uncomment the next line if you don't want to read from saved data
        raise ValueError('A very bad thing happened.')
        
        ### read from saved data, if available
        new_coefficients = np.load(savepath + "-coeff.npz")['arr_0']
        new_combinations = np.load(savepath + "-combs.npz")['arr_0']
    except:
        ### if saved data is not available, run this block
        
        new_coefficients = np.copy(coefficients)
        combinations = np.array([list(c) + ([1, 1, 0, 0] if ph_symmetrise_factor == 2 else [1, 1]) for c in combinations])
        new_combinations = np.copy(combinations).tolist()
        args = [[(source_comb, coefficients[(combinations == source_comb).all(axis=1).nonzero()][0]) for source_comb in combinations],
                new_combinations, new_coefficients, q_pos, alpha, ph_symmetrise_factor]
        new_combinations, new_coefficients = get_renorm_per_comb(args)

        new_coefficients /= np.linalg.norm(new_coefficients)
            
        ### Save (with compression!)
        np.savez_compressed(savepath + "-coeff.npz", new_coefficients)
        np.savez_compressed(savepath + "-combs.npz", new_combinations)
    return new_coefficients, new_combinations


def get_init_weight(args):
    comb, total_num, c_all, Xgs = args
    id_bath = tensor([identity(2)] * (total_num - 1))
    classicalState = tensor([basis(2,0)] * total_num)
    state_gen_op = 1
    for i, occ in enumerate(comb):
        if i == 0:
            state_gen_op = state_gen_op * (tensor(sigmam(), id_bath) if occ == 0 else 1)
        else:
            state_gen_op = state_gen_op * (c_all[i-1].dag() if occ == 1 else 1)
    classicalState = state_gen_op * classicalState
    weight = np.real(Xgs.dag() * classicalState)[0][0]
    if np.round(weight, 5) != 0:
        return comb, 2 * (comb[0] - 0.5) * abs(weight)
    else: 
        return
    
    
def init_wavefunction(J, D, num_in, ph_symmetrise_factor):
    """ Generates the initial wavefunction at the fixed point.
    Returns the coefficients associated with it and the list
    of all possible occupancies for each of the members.
    No IOMS are taken into account at this point."""
    
    total_num = 1 + 2 * num_in
    c_all = [tensor([identity(2)] + [sigmaz()]*i + [destroy(2)] + [identity(2)] * (total_num - 1 - i - 1)) for i in range(total_num - 1)]
    num_kup = [c.dag() * c for c in c_all[::2]]
    num_kdn = [c.dag() * c for c in c_all[1::2]]
    sigmadz = tensor([sigmaz()] + [identity(2)] * (total_num - 1))
    sigmadp = tensor([sigmap()] + [identity(2)] * (total_num - 1))
    sigmadm = sigmadp.dag()
    
    c0up = sum(c_all[::2])
    c0dn = sum(c_all[1::2])
    sigma0z = c0up.dag() * c0up - c0dn.dag() * c0dn
    c0up /= (sigma0z**2).tr()**0.25
    c0dn /= (sigma0z**2).tr()**0.25

    Ek = np.linspace(-D, D, num_in) #if ph_symmetrise_factor == 1 else np.linspace(-D, D, num_in)
    H = sum([Ek[i] * (num_kup[i] + num_kdn[i]) for i in range(len(num_kup))]) \
            + J * num_in * (0.25 * sigmadz * (c0up.dag() * c0up - c0dn.dag() * c0dn)
            + 0.5 * sigmadp * c0dn.dag() * c0up
            + 0.5 * sigmadm * c0up.dag() * c0dn
        )
    
    E, X = H.eigenstates()
    E = np.round(E, 7)

        
    Xgs_q = X[0] 
    assert (Xgs_q.dag() * sigmadz * Xgs_q)[0][0] == 0
    assert False not in [(Xgs_q.dag() * (num_kup[i] - num_kdn[i]) * Xgs_q)[0][0] == 0 for i in range(len(num_kup))]
    
    args = [[comb, total_num, c_all, Xgs_q] for comb in product([1,0], repeat=total_num)]
    results = list(tqdm(Pool().map(get_init_weight, args), total=2**total_num))
    combinations = [r[0] for r in results if r!= None]
    coefficients = [r[1] for r in results if r!= None]
    coefficients /= np.linalg.norm(coefficients)
    
    return np.round(coefficients, 12), combinations


def get_tensorRG(J0_by_D0, num_in, num_out, ph_symmetrise, computables):
    ph_symmetrise_factor = 2 if ph_symmetrise == True else 1
    D, J = get_RG_flow(J0_by_D0)
    # num_in = ph_symmetrise_factor * num_in
    num_in_start = num_in
    print ("Starting with", num_in,"k-states in the bath.") 
    omega = -D0/2
    
    ### ensure that there are enugh RG steps in J and D
    ### to recouple all IOMS.
    assert len(J) >= num_out + 1
    
    ### obtain the initial coefficients and set of occupancies
    savepath = "./MERGCoeffs/{:.0f},{:.0f},{:.10f},{:.10f},{:.10f},{:.10f}".format(round(D[0]/deltaD), 1 + 2 * num_in, J0_by_D0 * D0, D0, deltaD, omega)
    try:
        raise ValueError('A very bad thing happened.')
        coefficients = np.load(savepath + "-coeff.npz")['arr_0']
        combinations = np.load(savepath + "-combs.npz")['arr_0']
    except:
        coefficients, combinations = init_wavefunction(J[0], D[0], num_in, ph_symmetrise_factor)
        os.makedirs("./MERGCoeffs", exist_ok=True)
        np.savez_compressed(savepath + "-coeff.npz", coefficients)
        np.savez_compressed(savepath + "-combs.npz", combinations)
    
    ### save the initial set of coefficients and store the savepath
    ### in the list of savepaths that will be finally returned.
    coefficients_arr = [coefficients]
    savepaths = []
    savepaths.append(savepath)
    
    computations = {computable: [] for computable, members, _, _ in computables}
    ### initial the next set of coefficients and loop through 
    ### the RG flow to generate the tensor network RG.
    
    new_combinations = np.copy(combinations)
    for Ji, Di in tqdm(zip(J[1: num_out + 1], D[1: num_out + 1]), total=num_out, desc="Applying inverse unitaries"):
        total_num = 1 + 2 * num_in + 2 * ph_symmetrise_factor
        alpha = Ji / (omega - Di/2 + Ji/4)
        savepath = "./MERGCoeffs/{:.0f},{:.0f},{:.10f},{:.10f},{:.10f},{:.10f}".format(round(D[0]/deltaD), total_num, Ji, Di, deltaD, omega)

        ### get the next set of coefficients and append the savepath to access them later.
        coefficients, combinations = get_renorm(coefficients, combinations, savepath, total_num, alpha, ph_symmetrise_factor)
        savepaths.append(savepath)
        
        ### increase the number of entangled members.
        num_in += ph_symmetrise_factor
            
    x_arr = np.arange(num_in_start, num_in_start + ph_symmetrise_factor * num_out + 1, ph_symmetrise_factor)
    for computable, members, ylabel, savename in computables:
        if computable == "VNE":
            args = [[savepath, members] for savepath in savepaths]
            rho_k_arr = [rho_k for _, rho_k in tqdm(Pool(1).imap(get_rhok_perstep, args), total=len(savepaths))]
            computations[computable] = [entropy_vn(rho_k) for rho_k in rho_k_arr]

        if computable in ["I2", "distance"]:
            assert len(members) == 2
            args1 = [[savepath, members[0]] for savepath in savepaths]
            rho_k1_arr = [rho_k for _, rho_k in tqdm(Pool().imap(get_rhok_perstep, args1), total=len(savepaths))]
            S1 = [entropy_vn(rho_k) for rho_k in rho_k1_arr]
            args2 = [[savepath, members[1]] for savepath in savepaths]
            rho_k2_arr = [rho_k for _, rho_k in tqdm(Pool().imap(get_rhok_perstep, args2), total=len(savepaths))]
            S1 = [entropy_vn(rho_k) for rho_k in rho_k2_arr]
            args12 = [[savepath, list(members[0]) + list(members[1])] for savepath in savepaths]
            rho_k12_arr = [rho_k for _, rho_k in tqdm(Pool().imap(get_rhok_perstep, args12), total=len(savepaths))]
            S12 = [entropy_vn(rho_k) for rho_k in rho_k12_arr]
            I2 = np.array(S1) + np.array(S2) - np.array(S12)
            if computable == "I2":
                computations[computable] = I2
            else:
                computations[computable] = np.log(np.log(2**(len(members[0]) + len(members[1]) - 1)) / I2)

        if computable == "Qtemp":
            args = [[savepath, members] for savepath in savepaths]
            computations[computable] = list(tqdm(Pool().imap(get_Qtemp_perstep, args), total=len(savepaths)))
            
        if computable == "distribution":
            variance = []
            for savepath in savepaths:
                coefficients = np.load(savepath + "-coeff.npz")['arr_0']
                variance.append(np.var(abs(coefficients)))
            computations[computable] = variance
            (a, b, c, d, e),_ = scipy.optimize.curve_fit(fit_func, np.arange(num_in_start, num_in_start + num_out + 1), variance)
            
        if computable == "variance_rhok":
            args = [[savepath, members] for savepath in savepaths]
            rho_k_arr = [rho_k for _, rho_k in tqdm(Pool().imap(get_rhok_perstep, args), total=len(savepaths))]
            computations[computable] = [np.var(rho_k.diag()) for rho_k in rho_k_arr]
            
        if computable == "elements_rhok":
            args = [[savepath, members] for savepath in savepaths]
            data_arr = [data for data in tqdm(Pool().imap(get_rhok_perstep, args), total=len(savepaths))]
            for x, data in zip(x_arr, data_arr):
                for i,rho_ki in enumerate(np.array(data[1].diag()).flatten()):
                    plt.scatter(x, rho_ki, color=cols[i], marker=markers[i])
            [plt.scatter([], [], color=cols[i], marker=markers[i], label=''.join(map(str, data[0][i]))) for i in range(2**len(members))]
            plt.xlabel(r"no. of $k-$states in the c-bath")
            plt.ylabel(ylabel)
            plt.legend()
            plt.savefig(savename, bbox_inches='tight')
            plt.show()
            computations[computable] = []
            
        if computable == "S_kz":
            assert len(members) == 1
            for savepath in savepaths:
                coefficients = np.load(savepath + "-coeff.npz")['arr_0']
                combinations = np.load(savepath + "-combs.npz")['arr_0']
                # print (coefficients)
                # print (combinations[:,members[0]] == 1)
                computations[computable].append(0.5 * sum(coefficients[combinations[:,members[0]] == 1]**2)
                                                - 0.5 * sum(coefficients[combinations[:,members[0] + 1] == 1]**2))
            
        if len(computations[computable]) > 0:
            print (computations[computable])
            plt.scatter(x_arr, computations[computable])
            plt.xlabel(r"no. of $k-$states in the c-bath")
            plt.ylabel(ylabel)
            if savename: plt.savefig(savename, bbox_inches='tight')
            plt.show()
    return computations


def get_rhok_perstep(args):
    """Returns the VNE at a given RG step."""

    savepath, members = args
    ### Access the coefficients data for the given RG step.
    coefficients = np.load(savepath + "-coeff.npz")['arr_0']
    combinations = np.load(savepath + "-combs.npz")['arr_0']
    print (min(np.abs(coefficients)))

    ### Create the reduced density matrix by adding 
    ### the pertinent coefficients.
    rho_reduced = np.zeros([2**len(members)] * 2)
    members_combs = [list(comb) for comb in product([1,0], repeat=len(members))]

    for i,combination in enumerate(np.copy(combinations)):
        x_index = list(members_combs).index(list(combination[members]))
        rho_reduced[x_index][x_index] += coefficients[i]**2
        new_comb = combination[::]
        for y_index, mem_comb in enumerate(members_combs):
            if y_index == x_index: continue
            new_comb[members] = mem_comb
            if list(new_comb) in [list(c) for c in combinations]:
                j = [list(c) for c in combinations].index(list(new_comb))
                rho_reduced[x_index][y_index] += coefficients[i] * coefficients[j]
    rho_red_qutip = Qobj(rho_reduced, dims = [[2] * len(members)] * 2).unit()
    
    return members_combs, rho_red_qutip


def get_Qtemp_perstep(args):
    """Returns the VNE at a given RG step."""

    savepath, member = args
    ### Access the coefficients data for the given RG step.
    coefficients = np.load(savepath + "-coeff.npz")['arr_0']
    combinations = np.load(savepath + "-combs.npz")['arr_0']

    ### Create the reduced density matrix by adding 
    ### the pertinent coefficients.
    rho_reduced = np.zeros([2,2])
    
    
    rho_reduced[0][0] = sum(coefficients[combinations[:,member] == 0]**2)
    rho_reduced[1][1] = sum(coefficients[combinations[:,member] == 1]**2)

    ### Diagonalise reduced DM and return VNE.
    
    eigvals = np.linalg.eigvalsh(rho_reduced)

    return np.log(abs(rho_reduced[1][1] / rho_reduced[0][0]))

In [100]:
# computations = get_tensorRG(0.25, 1, 2, [['VNE', [1], r"$S_\text{EE}(k_1)$", "VNE_k.pdf"],
#                                          ['I2', [[0], [1,2]], r"$I_2(d:k_1)$", "I2_dk.pdf"],
#                                          ['I2', [[3,4], [1,2]], r"$I_2(k_2:k_1)$", "I2_kk.pdf"],
#                                         ])
# computations = get_tensorRG(0.05, 1, 30, False, [])
computations = get_tensorRG(0.1, 3, 4, True, [
                                        # ['VNE', [1], r"$S_\text{EE}(k_1)$", "VNE_k.pdf"],
                                        # ['VNE', [2], r"$S_\text{EE}(k_1)$", "VNE_k.pdf"],
                                        ['S_kz', [1], r"$\braket{S_{k1}^z}$", ""],
                                        ])

Starting with 3 k-states in the bath.


  0%|          | 0/128 [00:00<?, ?it/s]

Applying inverse unitaries:   0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/66 [00:00<?, ?it/s]

  0%|          | 0/454 [00:00<?, ?it/s]

  0%|          | 0/4150 [00:00<?, ?it/s]

KeyboardInterrupt: 

## **Benchmarks against a more "brute-force" approach**

In [ ]:
def bruteForceMERG(J0_by_D0, num_in, num_out):
    omega = -D0/2
    num_in_start = num_in
    D, J = get_RG_flow(J0_by_D0)
    assert len(J) >= num_out + 1
    total_dim = 1 + 2 * (num_in + num_out)
    c_all = [tensor([identity(2)] + [sigmaz()]*(i) + [destroy(2)] + [identity(2)]*(total_dim - i - 2)) for i in range(total_dim - 1)]
    Sdz = 0.5 * tensor([sigmaz()] +  [identity(2)] * (total_dim - 1))
    Sdp = tensor([sigmap()] +  [identity(2)] * (total_dim - 1))
    Sdm = Sdp.dag()
    vacuum = tensor([basis(2,0)]*total_dim)
    # init_state = 0
    # for k_pos in range(0, 2 * num_in, 2):
    #     init_state += c_all[k_pos + 1].dag() * vacuum
    #     init_state += - Sdm * c_all[k_pos].dag() * vacuum
    # gstates = [init_state.unit()]
    Ek = np.linspace(-D[0], D[0], num_in + num_out) #if ph_symmetrise_factor == 1 else np.linspace(-D, D, num_in)
    c0up = sum(c_all[::2]) / (0.5 * (total_dim - 1))
    c0dn = sum(c_all[1::2]) / (0.5 * (total_dim - 1))
    H = sum([Ek[int(i/2)] * cop.dag() * cop for i, cop in enumerate(c_all)]) \
            + J[0] * (Sdz * 0.5 * (c0up.dag() * c0up - c0dn.dag() * c0dn)
                   + 0.5 * Sdp * c0dn.dag() * c0up
                   + 0.5 * Sdm * c0up.dag() * c0dn
                  )
    
    E, Xgs_q = H.groundstate()
    print (Xgs_q)
    # print (E, Xgs_q.dag() * Sdz * Xgs_q)
    ref_state1 = Sdm * c_all[0].dag() * c_all[2].dag() * c_all[5].dag() * vacuum
    ref_state2 = c_all[1].dag() * c_all[3].dag() * c_all[4].dag() * vacuum
    print (ref_state1.dag() * Xgs_q)
    print (ref_state2.dag() * Xgs_q)
    return
    ref_state1 = c_all[3].dag() * vacuum
    ref_state2 = Sdm * c_all[2].dag() * vacuum
    ref_state3 = c_all[5].dag() * vacuum
    ref_state4 = Sdm * c_all[4].dag() * vacuum
    ref_state5 = Sdm * c_all[0].dag() * vacuum
    ref_state6 = c_all[1].dag() * vacuum

    for Ji, Eq in tqdm(zip(J[1:num_out+1], D[1:num_out+1]), total=num_out):
        q_pos = 2 * num_in
        alpha = Ji / (omega - Eq/2 + Ji/4)
        eta = 0
        eta_dag = 0
        for k_pos in range(0, q_pos, 2):
            for beta in [1, -1]:
                beta_pos = 0 if beta == 1 else 1
                eta += alpha * Sdz * beta * c_all[k_pos + beta_pos].dag() * c_all[q_pos + beta_pos]
                eta += alpha * [Sdp, Sdm][beta_pos] * c_all[k_pos + 1 - beta_pos].dag() * c_all[q_pos + beta_pos]
                eta_dag += alpha * Sdz * beta * c_all[q_pos + beta_pos].dag() * c_all[k_pos + beta_pos]
                eta_dag += alpha * [Sdp, Sdm][1 - beta_pos] * c_all[q_pos + beta_pos].dag() * c_all[k_pos + 1 - beta_pos]
        gstates.append(((1 + eta + eta_dag) * gstates[-1]).unit())

        num_in += 1
    
    vne_arr = []
    I2_arr = []
    for state in tqdm(gstates):
        rho = state * state.dag().unit()
        rho_k = rho.ptrace([1])
        rho_d = rho.ptrace([0])
        rho_dk = rho.ptrace([0, 1])
        SEE_k = entropy_vn(rho_k)
        SEE_d = entropy_vn(rho_d)
        SEE_dk = entropy_vn(rho_dk)
        I2_dk = SEE_k + SEE_d - SEE_dk
        vne_arr.append(SEE_k)
        I2_arr.append(I2_dk)                   
    print (vne_arr)
    print (I2_arr)

    # computations = get_tensorRG(J0_by_D0, num_in_start, num_out + 5, [['VNE', [1], r"$S_\text{EE}(k_1)$", "VNE_k.pdf"],
    #                                                                   ['I2', [[0], [1]], r"$I_2(d:k_1)$", "I2_dk.pdf"]])
    # plt.scatter(range(num_in_start, num_in_start + num_out + 1), vne_arr, color=cols[1], zorder=99, label="slower approach")
    # plt.plot(range(num_in_start, num_in_start + len(computations['VNE'])), computations['VNE'], label="faster approach")
    # plt.ylabel(r"$S_\text{{EE}}(k)$")
    # plt.xlabel(r"no. of $k-$states in the c-bath")
    # plt.text(0.2, 0.2, r"$n_\text{{IR}}={:.0f}$".format(num_in_start), bbox=bbox, horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes)
    # plt.legend()
    # plt.savefig("comparison_VNE_{:.0f}.pdf".format(num_in_start), bbox_inches='tight')
    # plt.show()
    # plt.scatter(range(num_in_start, num_in_start + num_out + 1), I2_arr, color=cols[1], zorder=99, label="slower approach")
    # plt.plot(range(num_in_start, num_in_start + len(computations['I2'])), computations['I2'], label="faster approach")
    # plt.ylabel(r"$I_2(d:k)$")
    # plt.xlabel(r"no. of $k-$states in the c-bath")
    # plt.text(0.2, 0.2, r"$n_\text{{IR}}={:.0f}$".format(num_in_start), bbox=bbox, horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes)
    # plt.legend()
    # plt.savefig("comparison_I2_{:.0f}.pdf".format(num_in_start), bbox_inches='tight')
    # plt.show()

bruteForceMERG(0.1, 3, 0)
# bruteForceMERG(0.1, 3, 4)